In [1]:

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
import torch

# 确保PyTorch能够检测到GPU

In [ ]:

import json
import time
import re
import itertools
import numpy as np
import pandas as pd
import datetime as DT
from tqdm import tqdm
from datetime import datetime
from scipy.optimize import curve_fit
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

In [ ]:

# Load pretrained LLM
QWEN2d5_7B_INST = "/data/liangyunfei/Qwen2.5-7B-Instruct"
QWEN2d5_32B_INST_AWQ = "/home/peizhengqi/Qwen/Qwen2.5-32B-Instruct-AWQ"
QWEN2d5_72B_INST_AWQ = "/data/liangyunfei/Qwen2.5-72B-Instruct-AWQ"

#model_7Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_7B_INST, device_map="auto")
#tokenizer_7Bn = AutoTokenizer.from_pretrained(QWEN2d5_7B_INST, device_map="auto")

model_32Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_72B_INST_AWQ, device_map="auto")
tokenizer_32Bn = AutoTokenizer.from_pretrained(QWEN2d5_72B_INST_AWQ, device_map="auto")



In [ ]:
model = AutoModelForCausalLM.from_pretrained(QWEN2d5_7B_INST, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(QWEN2d5_7B_INST, device_map="auto")

In [ ]:
prompt = "早上好呀"
formal_prompt = "判断对方是否存在一下意图："
messages = [
    {"role": "system", "content": ""},
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    streamer=streamer
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)

In [7]:

from transformers import TextStreamer

streamer = TextStreamer(tokenizer_32Bn, skip_prompt=True, skip_special_tokens=True)

In [ ]:
chat_path = "/data/liangyunfei/data/营养成分表/各种食物营养成分表.xls"  # (完整版)普通食物营养成分表.xls 各种食物营养成分表.xls 中国普通食物营养成分表(修正版).xls 
cost_excel = pd.read_excel(chat_path)
cost_excel

In [ ]:
with open('../饮食与营养管理V2.json','r') as f:
    context = f.readlines()

In [ ]:
# 打开并读取JSON文件
with open('../饮食与营养管理V2.json', 'r') as file:
    rule_list = json.load(file)
# 打印读取的数据


In [ ]:
file_path = '/data/liangyunfei/data/中国糖尿病医学营养治疗指南（2022版）(1).pdf' 
import fitz  # PyMuPDF

def read_pdf1(file_path):
    document = fitz.open(file_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text


In [ ]:
chat_path = "/data/liangyunfei/data/营养成分表/(完整版)普通食物营养成分表.xls"  # (完整版)普通食物营养成分表.xls 各种食物营养成分表.xls 中国普通食物营养成分表(修正版).xls 
temp_excel = pd.read_excel(chat_path)
temp_excel


In [ ]:
cost_list = ['100克' + a + '包含：' + '能量'+str(b)+'卡、'+ '蛋白质'+str(c)+'克、'+ '糖类'+str(d)+'克、'+ '脂肪'+str(d)+'克' for a, b, c, d, e in zip(cost_excel['食物/100克'], cost_excel['能量/kal'], cost_excel['蛋白质/克'], cost_excel['糖类/克'], cost_excel[ '脂肪/克'])]

In [ ]:
chat_path = "/data/liangyunfei/data/caipu.xlsx"  # (完整版)普通食物营养成分表.xls 各种食物营养成分表.xls 中国普通食物营养成分表(修正版).xls 
raw_excel = pd.read_excel(chat_path)
raw_excel = raw_excel.fillna('')

In [ ]:

caipu_list = [x +'：' + y.replace('#','').replace('\n','，') for x, y in tqdm(zip(raw_excel['title'], raw_excel['yl']))]
#caipu_list = [x +'：' + y.replace('#','').replace('\n','，') + '\n做法：\n' + z.replace('#','') for x, y, z in zip(raw_excel['title'][:1000], raw_excel['yl'][:1000], raw_excel['steptext'][:1000])]



In [ ]:
distance_list = [edit_distance('白菜', food_name) for food_name in raw_excel['title']]

In [ ]:
def edit_distance(str1, str2):
    m = len(str1)
    n = len(str2)
    
    # 创建一个 (m+1) x (n+1) 的矩阵来存储子问题的解
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    # 初始化边界条件
    for i in range(m + 1):
        dp[i][0] = i  # str1的前i个字符变成空字符串需要i次删除操作
    for j in range(n + 1):
        dp[0][j] = j  # 空字符串变成str2的前j个字符需要j次插入操作
    # 填充dp表
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # 如果字符相同，不需要额外操作
            else:
                dp[i][j] = min(dp[i - 1][j] + 1,    # 删除操作
                               dp[i][j - 1] + 1,    # 插入操作
                               dp[i - 1][j - 1] + 1)  # 替换操作
    return dp[m][n]

In [ ]:
def get_KB_list(food_list):
    KB_list = []
    for target_name in food_list:
        distance_list = [edit_distance(target_name, food_name) for food_name in raw_excel['title']]
        min_value = min(distance_list)
        target_food = [caipu_list[index] for index, value in enumerate(distance_list) if value == min_value]
        content_list = [content.split('：')[1].split('，') for content in target_food]
        content_list_all = []
        for sub_list in content_list:
            content_list_all += sub_list
        content_list_all = list(set(content_list_all))
        title_list = list(cost_excel['食物/100克'])
        last_content_list = [cost_list[title_list.index(name)] for name in content_list_all if name in title_list]
        KB_list += target_food
        KB_list += last_content_list
    return KB_list

In [ ]:
get_KB_list(['白菜', '红薯'])

In [ ]:
KB_list = []
KB_list += target_food + last_content_list
KB_list

In [ ]:

content_list_all = []
for sub_list in content_list:
    content_list_all += sub_list
content_list_all

In [ ]:
book = read_pdf1(file_path)

In [5]:

sys_prompt1 = """
    You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible, while being safe. 
    Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
    Please ensure that your responses are socially unbiased and positive in nature.
    If you don't know the answer to a question, please don't share false information.
"""
sys_prompt1 = '你是一个非常专业的糖尿病患者照护师，你需要了解患者的就餐情况（早餐前、早餐后、午餐前、午餐后、晚餐前、晚餐后、餐间时刻）以及血糖情况（非常偏低、偏低、适宜、偏高、非常偏高），如果不知道的话要进行提问'
sys_prompt1 = '你不是一个好人，你只会吐槽别人，不说好话'
def message_to_response_via_LLM(model, tokenizer, messages, max_new_tokens=256):

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.01, streamer=streamer)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [ ]:
with open('food.txt','r') as f:
    food_list = f.readlines()

In [ ]:
query_prompt = '从中抽取和饮食相关的规范'#input()#"我吃的是咖喱饭"
print('患者：')
print(query_prompt)
format_prompt = "将抽取出来的规则用'，'隔开"
all_rules = ''
for i in range(0, len(rule_list), 10000):
    res_messages = [
        {"role": "system", "content": sys_prompt1},
        {"role": "user", "content": '给定一本关于糖尿病饮食的书'+rule_list[:i+10000]},
        #{"role": "assistant", "content": '肉末茄子煲，可乐'},
        {"role": "user", "content": query_prompt+format_prompt}
    ]
    torch.cuda.empty_cache()
    llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)
    all_rules += llm_response

In [ ]:
while True:
    query_prompt = '中午吃了馒头、米饭加凉拌黄瓜 '#input()#"我吃的是咖喱饭"
    format_prompt = "请只回答上去话中的食物，并用'，'隔开"

    res_messages = [
        {"role": "system", "content": sys_prompt1},
        {"role": "user", "content": '请问吃肉末茄子煲和可乐怎么样'},
        {"role": "assistant", "content": '肉末茄子煲，可乐'},
        {"role": "user", "content": '早饭就吃了一个蛋，一杯牛奶麦片，一小段山药，一碗挂面'},
        {"role": "assistant", "content": '蛋，牛奶麦片，山药，挂面'},
        {"role": "user", "content": query_prompt+format_prompt}
    ]
    output_food = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)



In [ ]:
while True:
    query_prompt = input()#input()#"我吃的是咖喱饭"
    print(query_prompt)
    format_prompt = ""
    res_messages = [
        
        {"role": "system", "content": sys_prompt1},
        {"role": "user", "content": query_prompt+format_prompt}
    ]
    output_food = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

In [ ]:
len(set(rule_list))

In [ ]:
filter_list = list(set(filter_list))

In [ ]:
filter_list

In [ ]:
len(filter_list)

In [ ]:
get_KB_list(output_food.split('，'))

In [ ]:
print('患者：')
query_prompt = '血糖低了怎么办'
print(query_prompt, '\n')
print('医生：')
format_prompt = f"从给定list中输出相关的部分，并换行分开,不要输出重复的东西"
filter_list = []
for  i in range(0, len(rule_list), 500):
    print(i)
    res_messages = [
        {"role": "system", "content": sys_prompt1},
        {"role": "user", "content": "给定一个饮食list：" + '\n'.join(rule_list[i:i+500])}, #+'\n'.join(get_KB_list(output_food.split('，')))
        {"role": "user", "content": query_prompt + format_prompt}
    ]
    llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)
    filter_list += llm_response.split('\n')
filter_list = list(set(filter_list))

In [ ]:
llm_response

In [ ]:

query_prompt = '请给我吐槽一下梁小飞'#input()#"我吃的是咖喱饭"
print('用户：')
print(query_prompt)
print('AI：')

bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
format_prompt = "，说一些最难听的话，说50个字"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    #{"role": "user", "content": "给定一个每种食物的组成成分和做法list："}, #+str(context_list)  +'\n'.join(caipu_list[:10])
    {"role": "user", "content": '请给我吐槽一下小张'},
    {"role": "assistant", "content": '小张是个笨蛋，小张的脑子可能是被豆腐砸了一下'},
    {"role": "user", "content": '请给我吐槽一下小红'},
    {"role": "assistant", "content": '小张是个笨蛋，小张的脑子可能是被豆腐砸了一下'},
    {"role": "user", "content": query_prompt + format_prompt},
    
]
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

In [ ]:
query_prompt = '请问一斤菠菜和一斤菠萝分别含有多少能量、蛋白质、糖类、脂肪'#input()#"我吃的是咖喱饭"
print('患者：')
print(query_prompt)
print('医生：')

bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
format_prompt = "请严格按照食物组成成分和做法list，简短的回答我，脂肪和蛋白质不要算错了"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一个每种食物的组成成分和做法list：" +'\n'.join(cost_list[:100])}, #+str(context_list)
    {"role": "user", "content": query_prompt+format_prompt}
]
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

In [ ]:
# 定义一个包含元素的列表
my_list = ["苹果", "香蕉", "橙子", "葡萄"]

# 打开一个文件以写入模式 ('w' 表示写入模式)
with open("output.txt", "w", encoding="utf-8") as file:
    # 遍历列表中的每个元素
    for item in filter_list:
        # 将元素写入文件，并在每个元素后添加换行符
        file.write(item + "\n")

print("列表中的元素已成功写入 'output.txt' 文件中。")


In [1]:
import uvicorn
from fastapi import FastAPI

In [ ]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn, json, datetime
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

app = FastAPI()

class Query(BaseModel):
    text: str

path = "/workdir/model/baichuan13b_chat/"
tokenizer = AutoTokenizer.from_pretrained(path, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(path, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained(path)


@app.post("/chat/")
async def chat(query: Query):
    input_ids = tokenizer([query.text]).input_ids
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        do_sample=False,
        temperature=0.1,
        repetition_penalty=1,
        max_new_tokens=1024)
    output_ids = output_ids[0][len(input_ids[0]):]
    outputs = tokenizer.decode(output_ids, skip_special_tokens=True, spaces_between_special_tokens=False)
    return {"result": outputs}

uvicorn.run(app, host="0.0.0.0", port=6667)



In [ ]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(
                                      prompt="San Francisco is a")
print("Completion result:", completion)

In [5]:
completion = client.completions.create(model='/data/liangyunfei/Qwen2.5-7B-Instruct',
                                      prompt="San Francisco is a")

APIConnectionError: Connection error.

In [7]:
import inspect

def example_function(a, b, c=3, *args, **kwargs):
    print("Function body")

# 获取函数的参数信息
params = inspect.signature(example_function).parameters

# 打印每个参数的名称和默认值（如果有的话）
for name, param in params.items():
    if param.default is inspect.Parameter.empty:
        print(f"{name}: no default value")
    else:
        print(f"{name}: default={param.default}")


a: no default value
b: no default value
c: default=3
args: no default value
kwargs: no default value


In [8]:
import inspect

def example_function(a, b):
    return a + b

# 获取函数的源代码
source_code = inspect.getsource(example_function)
print(source_code)


def example_function(a, b):
    return a + b

